## Getting Started
Loading eye movement dataset with pymovements

In [3]:
from __future__ import annotations

import os
import pymovements as pm

In [2]:
from dataclasses import dataclass, field
from typing import Any

import polars as pl
from pymovements.dataset.dataset_definition import DatasetDefinition
from pymovements.dataset.dataset_library import register_dataset
from pymovements.gaze.experiment import Experiment


@dataclass
@register_dataset
class SBSATDataset(DatasetDefinition):
    name: str = "SB-SAT"

    has_files: dict[str, bool] = field(
        default_factory=lambda: {
            "gaze": True,
            "precomputed_events": False,
            "precomputed_reading_measures": False,
        }
    )

    mirrors: dict[str, tuple[str, ...]] = field(
        default_factory=lambda: {}
    )

    resources: dict[str, tuple[dict[str, str], ...]] = field(
        default_factory=lambda: {}
    )

    extract: dict[str, bool] = field(
        default_factory=lambda: {"gaze": False}
    )

    # Adjust these experiment parameters if the real SB-SAT setup differs
    experiment: Experiment = Experiment(
        screen_width_px=1280,
        screen_height_px=1024,
        screen_width_cm=38,
        screen_height_cm=30.2,
        distance_cm=68,
        origin="upper left",
        sampling_rate=1000,
    )

    # Regex: match any CSV ending in fixfinal.csv (e.g. 18sat_fixfinal.csv)
    filename_format: dict[str, str] = field(
        default_factory=lambda: {"gaze": r".*fixfinal\.csv"}
    )

    # If you don't need typed named groups from the filename, just keep this empty
    filename_format_schema_overrides: dict[str, dict[str, type]] = field(
        default_factory=lambda: {"gaze": {}}
    )

    # If you want each row to be a separate trial, you can specify which columns mark trials
    trial_columns: list[str] = field(
        default_factory=lambda: ["TRIAL_INDEX"]
    )

    # We'll treat CURRENT_FIX_DURATION as a time measure (in ms)
    time_column: str = "CURRENT_FIX_DURATION"
    time_unit: str = "ms"

    # The pixel coordinate columns
    pixel_columns: list[str] = field(
        default_factory=lambda: ["CURRENT_FIX_X", "CURRENT_FIX_Y"]
    )

    # If not renaming columns, keep this empty
    column_map: dict[str, str] = field(
        default_factory=lambda: {}
    )

    # Reading CSV files: specify columns, separator, and data types
    custom_read_kwargs: dict[str, dict[str, Any]] = field(
        default_factory=lambda: {
            "gaze": {
                "columns": [
                    "RECORDING_SESSION_LABEL",
                    "TRIAL_INDEX",
                    "CURRENT_FIX_X",
                    "CURRENT_FIX_Y",
                    "CURRENT_FIX_DURATION",
                ],
                "schema_overrides": {
                    "CURRENT_FIX_DURATION": pl.Float64,
                    "CURRENT_FIX_X": pl.Float64,
                    "CURRENT_FIX_Y": pl.Float64,
                },
                "separator": ",",  # SB-SAT fixfinal.csv likely comma-separated
                "null_values": "",
            },
        },
    )

In [3]:
# Initialize your dataset
dataset = pm.Dataset(
    definition=SBSATDataset(),  # Use your definition with name="SB-SAT"
    path="data/SB-SAT/fixation",  # The custom paths ignoring fill_name
)

dataset.load()

  0%|          | 0/1 [00:00<?, ?it/s]

In [212]:
# Ensure the data directory exists
os.makedirs('data/ToyDataset', exist_ok=True)

dataset = pm.Dataset(
    'ToyDataset',  # choose a public dataset from our dataset library
    path='data/ToyDataset',  # set up your local dataset path
)
dataset.download()  # download a public dataset from our dataset library
dataset.load()  # download the dataset

Using already downloaded and verified file: data\ToyDataset\downloads\pymovements-toy-dataset.zip
Extracting pymovements-toy-dataset.zip to data\ToyDataset\raw


  0%|          | 0/20 [00:00<?, ?it/s]

Calculate velocities on the fly

In [4]:
dataset.pix2deg()  # transform pixel coordinates to degrees of visual angle
dataset.pos2vel()  # transform positional data to velocity data

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

and extract events with different eye movements event extraction algorithms

In [5]:
dataset.detect('ivt')  # detect fixation using the I-VT algorithm
dataset.detect('microsaccades')  # detect saccades using the microsaccades algorithm

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Upload and Select Stimulus (Video or Image)
Upload the stimulus video or image file.

In [1]:
def create_eye_tracking_video(gaze_df, video_filename="eye_tracking_video.mp4", fps=30, width=1280, height=1024):
    """
    Creates a blank video with a duration matching the eye movement timestamps in the dataset.

    Parameters:
        gaze_df (pandas.DataFrame): The eye movement dataset containing 'time' column.
        video_filename (str): Name of the output video file.
        fps (int): Frames per second.
        width (int): Video width.
        height (int): Video height.
    """
    if "frame_idx" in gaze_df.columns:
        # If 'frame_idx' exists, use its max value to determine total frames
        total_frames = int(gaze_df["frame_idx"].max()) + 1
    else:
        # Otherwise, compute total frames based on timestamps
        min_time = gaze_df["time"].min() / 1000.0  # Convert ms → s
        max_time = gaze_df["time"].max() / 1000.0  # Convert ms → s
        duration = max_time - min_time  # Total video duration in seconds
        total_frames = int(duration * fps)

    # Ensure at least 1 frame is written
    total_frames = max(1, total_frames)

    print(f"Creating video of duration: {duration:.2f} sec ({total_frames} frames)")

    # Set up the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
    out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

    # Create white frames and write to video
    white_frame = np.ones((height, width, 3), dtype=np.uint8) * 255  # White background
    for _ in range(total_frames):
        out.write(white_frame)

    out.release()
    print(f"Video saved as {video_filename}")

In [4]:
import ipyfilechooser
import cv2
import numpy as np
from IPython.display import display

# Step 1: Create the file chooser widget
base_dir = os.getcwd()  # Start from the current working directory
while not base_dir.endswith("pymovements-videoreplay") and os.path.dirname(base_dir) != base_dir:
    base_dir = os.path.dirname(base_dir)  # Go up one level until we find the project root

default_path = os.path.join(base_dir, "data")

os.makedirs(default_path, exist_ok=True)

chooser = ipyfilechooser.FileChooser(default_path)
chooser.filter_pattern = ['*.mp4', '*.jpg', '*.jpeg', '*.png']  # Show video & image files
display(chooser)

# Step 2: Function to move the selected file or use the generated video
def save_uploaded_stimulus():
    selected_file_path = chooser.selected  # Get selected file path

    if selected_file_path is None:
        # If no video or image is uploaded, generate a white background video
        default_video = "white_background.mp4"
        if not os.path.exists(default_video):
            print(f"No stimulus uploaded. Generating default video with white background: {default_video}")
            create_eye_tracking_video(dataset.gaze[0].frame.to_pandas(), default_video)

        # Check if the generated image exists
        if os.path.exists(default_video):
            print(f"Using generated white background video: {default_video}")
            selected_file_path = default_video
        else:
            print("ERROR: No stimulus uploaded and failed to generate an image. Please upload an image or video.")
            return

    # Verify the file exists
    stimulus_name = os.path.basename(selected_file_path)
    if os.path.exists(selected_file_path):
        print(f"Stimulus saved successfully as '{stimulus_name}'!")
        print(f"File Size: {os.path.getsize(selected_file_path)} bytes")
    else:
        print("ERROR: Stimulus file was not saved correctly!")
        
    return selected_file_path    

FileChooser(path='C:\Users\Kirthan\IdeaProjects\pymovements-videoreplay\data', filename='', title='', show_hid…

In [5]:
# Step 3: After Selecting the File, Run:
selected_file_path = save_uploaded_stimulus()

Stimulus saved successfully as 'reading-dickens-1.png'!
File Size: 169398 bytes


## Initialize and Run VideoPlayer
Run the VideoPlayer to visualize eye-tracking data overlay.

In [17]:
import sys
import importlib

# Ensure 'src' is in sys.path
src_path = os.path.abspath(os.path.join('src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Import the module
import videoreplay.video_player

# Reload the module
importlib.reload(videoreplay.video_player)

# Now import the updated class
from videoreplay.video_player import VideoPlayer

# Initialize the VideoPlayer with the uploaded video
player = VideoPlayer(stimulus_path=selected_file_path,
                     dataset_path="data/SB-SAT/fixation",
                     recording_session="msd001")

Loading gaze data from: data\SB-SAT\fixation\18sat_fixfinal.csv


## Play Video with Gaze Overlay

In [18]:
player.play(speed=1.0)

## Navigate Through Fixations
Use 'n' for next fixation, 'p' for previous, 'q' to quit.

In [19]:
player.fixation_navigation()

## Export Gaze Replay as MP4
Save the replay with gaze overlay.

In [20]:
player.export_replay("gaze_replay_slow", 5)

Exporting gaze replay for an image stimulus...
Image-based replay exported as MP4: gaze_replay_slow.mp4
